In [ ]:
!nvcc --version
# UPDATE SPACY INSTALL TO MATCH CUDA VERSION
!pip install -U spacy
!pip install -U spacy[cuda111]
!python -m spacy download en_core_web_trf

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
     |████████████████████████████████| 6.0 MB 5.3 MB/s 
     |████████████████████████████████| 10.1 MB 40.4 MB/s 
     |████████████████████████████████| 451 kB 44.5 MB/s 
     |████████████████████████████████| 181 kB 51.7 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 628 kB 53.9 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thi

# Train/dev split (90/10)

In [ ]:
import srsly
import spacy
from spacy.training import docs_to_json, offsets_to_biluo_tags, biluo_tags_to_spans
from random import shuffle

# Data conversion

In [ ]:
# CONVERT TO IOB
train = srsly.read_json(r'/content/drive/MyDrive/4th year project/prodigy_train.json')

nlp = spacy.load('en_core_web_trf')
docs = []
for text, annot in train:
    doc = nlp(text)
    tags = offsets_to_biluo_tags(doc, annot['entities'])
    entities = biluo_tags_to_spans(doc, tags)
    doc.ents = entities
    docs.append(doc)

srsly.write_json(r'/content/drive/MyDrive/4th year project/prodigy_train_iob.json', [docs_to_json(docs)])

dev = srsly.read_json(r'/content/drive/MyDrive/4th year project/prodigy_dev.json')

docs = []
for text, annot in dev:
    doc = nlp(text)
    tags = offsets_to_biluo_tags(doc, annot['entities'])
    entities = biluo_tags_to_spans(doc, tags)
    doc.ents = entities
    docs.append(doc)

srsly.write_json(r'/content/drive/MyDrive/4th year project/prodigy_dev_iob.json', [docs_to_json(docs)])

In [ ]:
# CONVERT TO SPACY BINARY 
!python -m spacy convert "/content/drive/MyDrive/4th year project/prodigy_train_iob.json" "/content/drive/MyDrive/4th year project" -t spacy
!python -m spacy convert "/content/drive/MyDrive/4th year project/prodigy_dev_iob.json" "/content/drive/MyDrive/4th year project" -t spacy

✔ Generated output file (738 documents): /content/drive/MyDrive/4th
year project/prodigy_train_iob.spacy
✔ Generated output file (83 documents): /content/drive/MyDrive/4th year
project/prodigy_dev_iob.spacy


# Create spacy config

In [ ]:
!python -m spacy init fill-config "/content/drive/MyDrive/4th year project/bert_ner_config.cfg"
!python -m spacy debug data "/content/drive/MyDrive/4th year project/bert_ner_config.cfg"

[paths]
train = "/content/drive/MyDrive/4th year project/prodigy_train_iob.spacy"
dev = "/content/drive/MyDrive/4th year project/prodigy_dev_iob.spacy"
vectors = null
init_tok2vec = null

[system]
gpu_allocator = "pytorch"
seed = 0

[nlp]
lang = "en"
pipeline = ["transformer","ner"]
batch_size = 128
disabled = []
before_creation = null
after_creation = null
after_pipeline_creation = null
tokenizer = {"@tokenizers":"spacy.Tokenizer.v1"}

[components]

[components.ner]
factory = "ner"
incorrect_spans_key = null
moves = null
scorer = {"@scorers":"spacy.ner_scorer.v1"}
update_with_oracle_cut_size = 100

[components.ner.model]
@architectures = "spacy.TransitionBasedParser.v2"
state_type = "ner"
extra_state_tokens = false
hidden_width = 64
maxout_pieces = 2
use_upper = false
nO = null

[components.ner.model.tok2vec]
@architectures = "spacy-transformers.TransformerListener.v1"
grad_factor = 1.0
pooling = {"@layers":"reduce_mean.v1"}
upstream = "*"

[components.transformer]
factory = "transfor

# Training model 

In [ ]:
!python -m spacy train -g 0 "/content/drive/MyDrive/4th year project/bert_ner_config.cfg" --output "/content/drive/MyDrive/4th year project"

ℹ Saving to output directory: /content/drive/MyDrive/4th year
project
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-03-12 18:46:32,834] [INFO] Set up nlp object from config
[2022-03-12 18:46:32,847] [INFO] Pipeline: ['transformer', 'ner']
[2022-03-12 18:46:32,856] [INFO] Created vocabulary
[2022-03-12 18:46:32,858] [INFO] Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 441kB/s]
Downloading: 100% 878k/878k [00:00<00:00, 6.41MB/s]
Downloading: 100% 446k/446k [00:00<00:00, 3.17MB/s]
Downloading: 100% 1.29M/1.29M [00:00<00:00, 7.23MB/s]
Downloading: 100% 478M/478M [00:11<00:00, 42.6MB/s]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model

# Evaluate on unseen dev set

In [ ]:
!python -m spacy evaluate -g 0 "/content/drive/MyDrive/4th year project/model-best" "/content/drive/MyDrive/4th year project/prodigy_dev_iob.spacy" -o "/content/drive/MyDrive/4th year project/metrics.json" -dp "/content/drive/MyDrive/4th year project/visualisations"

ℹ Using GPU: 0

================================== Results ==================================

TOK     100.00
NER P   64.12 
NER R   70.59 
NER F   67.20 
SPEED   1609  


=============================== NER (per type) ===============================

          P       R       F
TYN   64.12   70.59   67.20

/usr/local/lib/python3.7/dist-packages/spacy/displacy/__init__.py:200: UserWarning: [W006] No entities to visualize found in Doc object. If this is surprising to you, make sure the Doc was processed using a model that supports named entity recognition, and check the `doc.ents` property manually if necessary.
  warnings.warn(Warnings.W006)
✔ Generated 25 parses as HTML
/content/drive/MyDrive/4th year project
✔ Saved results to /content/drive/MyDrive/4th year
project/metrics.json


In [ ]:
!python -m spacy evaluate -g 0 "/content/drive/MyDrive/4th year project/model-best" "/content/drive/MyDrive/4th year project/prodigy_dev_iob.spacy" -o "/content/drive/MyDrive/4th year project/metrics.json" -dp "/content/drive/MyDrive/4th year project/visualisations"